# Import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 6.8 MB/s 
     |████████████████████████████████| 182 kB 72.1 MB/s 
     |████████████████████████████████| 7.6 MB 74.6 MB/s 


In [ ]:
import tensorflow as tf
from transformers import AutoTokenizer
from transformers import TFGPT2LMHeadModel

# Load Tokenizer & Model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', pad_token='<pad>')
model = TFGPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2', from_pt=True)

Downloading:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Downloading:   0%|          | 0.00/513M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['lm_head.weight', 'transformer.h.9.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.6.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2LMHeadModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassifica

## 사전 학습된 모델 불러오기

In [ ]:
model =  TFGPT2LMHeadModel.from_pretrained('/content/drive/MyDrive/메인프로젝트(5조)/chat_bot/train/GPT/model_6.h5')

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at /content/drive/MyDrive/메인프로젝트(5조)/chat_bot/train/GPT/model_6.h5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


# Load Configfile

In [ ]:
from transformers import AutoConfig
config = AutoConfig.from_pretrained("skt/kogpt2-base-v2")

In [ ]:
import pandas as pd
import tqdm
import urllib.request

# Load Data

In [ ]:
train_path = '/content/drive/MyDrive/메인프로젝트(5조)/chat_bot/data/증강결과/호그와트_qna_origin_sim.xlsx'
val_path = '/content/drive/MyDrive/메인프로젝트(5조)/chat_bot/data/증강결과/호그와트_qna_orign_back_sim_backsim.xlsx'

train_df = pd.read_excel(train_path)
val_df = pd.read_excel(val_path)

question_list = train_df['question'].tolist()
answer_list = train_df['answer'].tolist()

In [ ]:
len(question_list), len(answer_list), len(train_df)

(5950, 5950, 5950)

# DataSet

In [ ]:
batch_size = 1 #@param {type:"slider", min:1, max:1000, step:1}

In [ ]:
def get_data():
  for question, answer in zip(question_list, answer_list):
    bos_token = [tokenizer.bos_token_id]
    eos_token = [tokenizer.eos_token_id]
    sent = tokenizer.encode('<question>' + question + '<answer>' + answer) 
    yield bos_token + sent + eos_token

In [ ]:
dataset = tf.data.Dataset.from_generator(get_data, output_types=tf.int32)
dataset = dataset.padded_batch(batch_size=batch_size, padded_shapes=(None,), padding_values=tokenizer.pad_token_id)

In [ ]:
for batch in dataset:
    print(batch)
    break

tf.Tensor(
[[42000     2  2030 27402 18226 15377  2032  7799  8135  8061 14992 14247
  14940  8311  2033  2030 19403  8364  8464 14177  2032  4023 14280  2070
  14460  8016 23567 18568 17959 14245 14589 13975  2025 16689  8031 25467
  20110 17959 14940  8311  2016     3 42000]], shape=(1, 43), dtype=int32)


In [ ]:
tokenizer.decode(batch[0])

'</s> [CLS] < question > 호그와트는 어떤 학교야? < answer > 만 11 ~ 18세까지의 청소년 마법사를 교육하는 7학년제 영국의 유일한 마법 학교야. [SEP] </s>'

In [ ]:
print(batch[0])

tf.Tensor(
[42000     2  2030 27402 18226 15377  2032  7799  8135  8061 14992 14247
 14940  8311  2033  2030 19403  8364  8464 14177  2032  4023 14280  2070
 14460  8016 23567 18568 17959 14245 14589 13975  2025 16689  8031 25467
 20110 17959 14940  8311  2016     3 42000], shape=(43,), dtype=int32)


In [ ]:
adam = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08)
steps = len(train_df) // batch_size
print("steps : ",steps, ", len(df) : ",len(train_df))

steps :  5950 , len(df) :  5950


# Training Model

In [ ]:
EPOCHS = 9 #@param {type:"slider", min:1, max:100, step:1}

for epoch in range(EPOCHS):
  epoch_loss = 0

  for batch in tqdm.tqdm_notebook(dataset, total=steps):
      with tf.GradientTape() as tape:
          result = model(batch, labels=batch)  # result랑 loss 값 확인
          loss = result[0]
          batch_loss = tf.reduce_mean(loss)
          
      grads = tape.gradient(batch_loss, model.trainable_variables)
      adam.apply_gradients(zip(grads, model.trainable_variables))
      epoch_loss += batch_loss / steps

  print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, epoch_loss))
  model.save_pretrained('/content/drive/MyDrive/메인프로젝트(5조)/chat_bot/train/GPT/model_{}.h5'.format(epoch + 2))


<ipython-input-38-8a92c0640412>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch in tqdm.tqdm_notebook(dataset, total=steps):


  0%|          | 0/5950 [00:00<?, ?it/s]

InvalidArgumentError: ignored

# Save Model

In [ ]:
model.save_pretrained('/content/drive/MyDrive/trans_koreng_project/KoGPT2/model_title_lyric_2')

# Active chatbot

In [ ]:
num_beams = 3 #@param {type:"slider", min:3, max:50, step:1}
min_length = 5 #@param {type:"slider", min:5, max:1000, step:1}
MAX_length = 100 #@param {type:"slider", min:1, max:10000, step:1}

def answer_predict(user_text):
  sent = '<quetion>' + user_text + '<answer>'
  input_ids = [tokenizer.bos_token_id] + tokenizer.encode(sent)
  input_ids = tf.convert_to_tensor([input_ids])
  output = model.generate(input_ids, max_length=MAX_length, do_sample=True, top_k=3)
  sentence = tokenizer.decode(output[0].numpy().tolist())
  title_predict = sentence.split('<answer>')[1].replace('</s>', '')
  return title_predict

In [ ]:
import time 
start = time.time()

answer = answer_predict('남자친구 있어?')
print(answer)

end = time.time()
print("작동시간:", f"{end - start:.5f} sec")

 남자 기숙사에 들어갈 수 없는데요?
작동시간: 1.85422 sec
